In [4]:
import os

In [5]:
%pwd

'c:\\Users\\5300\\Desktop\\image_classification\\research'

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Users\\5300\\Desktop\\image_classification'

In [32]:
# Add entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path
    

In [9]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories



In [34]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
            )
        return data_ingestion_config

In [18]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self)-> str:

        '''
        Fetch data from the url
        '''
        try:
            dataset_url = self.config.source_url
            zip_download_dir = self.config.local_data_file
            os.makedirs('artifacts/data_ingestion',exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} into {zip_download_dir}')

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id, zip_download_dir)
            logger.info(f'Successfully downloaded from {dataset_url} into file {zip_download_dir}')
        
        except Exception as e:
            raise e
    
    def extract_zip_file(self) -> None:
        '''
        zip_file_path: str
        Extracts the zip file into the data directory
        Function return None
        '''
        try:
            unzip_path = self.config.unzip_dir
            os.makedirs(unzip_path,exist_ok=True)
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)

        except Exception as e:
            raise e

In [36]:
# Pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-12-24 14:22:19,016: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-24 14:22:19,020: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-24 14:22:19,022: INFO: common: created directory at: artifacts]
[2023-12-24 14:22:19,024: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-24 14:22:19,026: INFO: 777272323: Downloading data from https://drive.google.com/file/d/1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3/view?usp=sharing into artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3
From (redirected): https://drive.google.com/uc?/export=download&id=1vlhZ5c7abUKF8xXERIw6m9Te8fW7ohw3&confirm=t&uuid=9e0cacdd-f8ac-428a-900e-9867b5c62504
To: c:\Users\5300\Desktop\image_classification\artifacts\data_ingestion\data.zip
  1%|          | 524k/57.7M [00:20<09:29, 101kB/s]

KeyboardInterrupt: 